<a href="https://colab.research.google.com/github/lorifin/PAL/blob/main/codir_data_slack_notion_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Installer les dépendances
!pip install openai matplotlib pandas slack_sdk notion-client

# 2. Import des librairies
import pandas as pd
import matplotlib.pyplot as plt
import openai
from slack_sdk import WebClient
from notion_client import Client
from datetime import datetime

# 3. Configurations
OPENAI_API_KEY = ""                    # Ta clé OpenAI               # Ton token Slack
SLACK_CHANNEL_ID = ""             # ID du canal Slack
NOTION_TOKEN = ""              # Token Notion API
NOTION_DATABASE_ID = ""  # ID de ta base Notion
SLACK_BOT_TOKEN = ""             # Bot User OAuth Token Slack
SLACK_CHANNEL = ""



# 4. Import CSV
df = pd.read_csv("data_codir.csv")
display(df)

# 5. Générer et sauvegarder le graphique
mois = df["Mois"]
ca = df["CA réalisé (€)"]
objectif = df["Objectif (€)"]
pipe = df["Pipe (€)"]

plt.figure(figsize=(8,5))
plt.plot(mois, ca, marker='o', label='CA Réalisé')
plt.plot(mois, objectif, marker='o', linestyle='--', label='Objectif')
plt.bar(mois, pipe, alpha=0.2, label='Pipe')
plt.title("Synthèse Commerciale Trimestrielle")
plt.ylabel("Montant (€)")
plt.legend()
plt.tight_layout()
plt.savefig("ventes_graph.png")
plt.close()

# 6. Générer la synthèse GPT
openai.api_key = OPENAI_API_KEY

prompt = f"""
Voici le tableau des indicateurs commerciaux du trimestre :
Mois : {', '.join(df['Mois'])}
CA réalisé : {', '.join(str(x) for x in df['CA réalisé (€)'])}
Objectif : {', '.join(str(x) for x in df['Objectif (€)'])}
Pipe : {', '.join(str(x) for x in df['Pipe (€)'])}
Opportunités : {', '.join(df['Opportunités'])}
Risques : {', '.join(df['Risques/Blocages'])}

Donne-moi :
1. Une synthèse Codir (en 3 phrases)
2. Les 2 points de vigilance et opportunités majeures
3. 2 actions prioritaires à piloter ce trimestre
"""

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=400,
    temperature=0.2,
)
synthese_codir = response.choices[0].message.content
print("==== Synthèse automatique Codir ====")
print(synthese_codir)

# 7. Envoi sur Slack (texte)
slack_client = WebClient(token=SLACK_BOT_TOKEN)
try:
    slack_client.chat_postMessage(
        channel=SLACK_CHANNEL_ID,
        text=f"Synthèse Codir automatique :\n{synthese_codir}"
    )
    print("✅ Synthèse envoyée sur Slack.")
except Exception as e:
    print(f"❌ Erreur Slack (texte) : {e}")

# 8. Envoi sur Slack (graphe)
try:
    slack_client.files_upload_v2(
        channel=SLACK_CHANNEL_ID,
        file="ventes_graph.png",
        title="Graphique Codir",
        initial_comment="Voici l’évolution des ventes 📈"
    )
    print("✅ Graphe envoyé sur Slack.")
except Exception as e:
    print(f"❌ Erreur Slack (graphe) : {e}")

# 9. Enregistrement de la synthèse dans Notion
notion = Client(auth=NOTION_TOKEN)
try:
    # ⚠️ Remplace "Nom" et "Synthèse" par les vrais noms de colonnes de ta base Notion
    page = notion.pages.create(
        parent={"database_id": NOTION_DATABASE_ID},
        properties={
            "Nom": {"title": [{"text": {"content": f"Synthèse Codir {datetime.now().strftime('%d/%m/%Y')}"} }]},
            "Synthèse": {"rich_text": [{"text": {"content": synthese_codir}}]}
            # Ajoute d'autres propriétés selon ta base (Date, etc.)
        }
    )
    print("✅ Synthèse enregistrée dans Notion.")
except Exception as e:
    print(f"❌ Erreur Notion : {e}")
